In [1]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

In [2]:
def plotEllipsoid(A, center):
	"""Plot Ellipsoid with A matrix and center"""
	# random select color
	color = tuple(np.random.randint(256, size=3) / 255)

	# find the rotation matrix and radii of the axes
	U, s, rotation = linalg.svd(A)
	radii = 1.0 / np.sqrt(s)

	u = np.linspace(0, 2 * np.pi, 100)
	v = np.linspace(0, np.pi, 100)

	x = radii[0] * np.outer(np.cos(u), np.sin(v))
	y = radii[1] * np.outer(np.sin(u), np.sin(v))
	z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

	for i, _ in enumerate(x):
		for j, _ in enumerate(x):
			[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center

	fig = plt.figure()
	ax = fig.add_subplot(111, projection='3d')
	ax.plot_wireframe(x, y, z, rstride=3, cstride=3, color=color, alpha=0.4)
	ax.scatter(center[0],center[1],center[2])
	ax.set_xlabel("X")
	ax.set_ylabel("Y")
	ax.set_zlabel("Z")
	plt.show()

def Ellipsoid_coordinates(A, center):
	# find the rotation matrix and radii of the axes
	U, s, rotation = linalg.svd(A)
	radii = 1.0 / np.sqrt(s)

	u = np.linspace(0, 2 * np.pi, 100)
	v = np.linspace(0, np.pi, 100)

	x = radii[0] * np.outer(np.cos(u), np.sin(v))
	y = radii[1] * np.outer(np.sin(u), np.sin(v))
	z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

	for i, _ in enumerate(x):
		for j, _ in enumerate(x):
			[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center
	return x,y,z

def plot_three_Ellipsoids(x1,y1,z1,x2,y2,z2,x3,y3,z3,c1,c2,c3):

	fig = plt.figure()
	ax = fig.add_subplot(111, projection='3d')

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x1, y1, z1, rstride=3, cstride=3, color="r", alpha=0.25)
	ax.scatter(c1[0],c1[1],c1[2], color = "r")

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x2, y2, z2, rstride=3, cstride=3, color="g", alpha=0.25)
	ax.scatter(c2[0],c2[1],c2[2], color = "g")

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x3, y3, z3, rstride=3, cstride=3, color="b", alpha=0.25)
	ax.scatter(c3[0],c3[1],c3[2], color = "b")
	ax.set_xlabel("X [m]")
	ax.set_ylabel("Y [m]")
	ax.set_zlabel("Z [m]")
	plt.show()

In [3]:
class Ellipsoid:
	"""Class to make ellipsoid."""

	@staticmethod
	def getMinVolEllipse(P, tolerance=0.01):
		"""Find the minimum volume of ellipsoid which holds all the points.

		based on code from https://github.com/minillinim/ellipsoid and modified.

		Parameters:
			P: numpy array of N-dimensional points like this:
				P = [[x, y, z, ...], <--- one point per line
					 [x, y, z, ...],
					 [x, y, z, ...]]

			tolerance: convergence criterion

		Returns:
			(center, radii, rotation)
		"""
		N, d = np.shape(P)
		d = float(d)

		# Q will the working array
		Q = np.vstack([np.copy(P.T), np.ones(N)])
		QT = Q.T

		# initializations
		err = 1.0 + tolerance
		u = (1.0 / N) * np.ones(N)

		# Using Khachiyan Algorithm
		while err > tolerance:
			V = np.dot(Q, np.dot(np.diag(u), QT))
			M = np.diag(np.dot(QT, np.dot(linalg.inv(V), Q)))
			k = np.argmax(M)
			maximum = M[k]
			step_size = (maximum - d - 1.0) / ((d + 1.0) * (maximum - 1.0))
			u_n = (1 - step_size) * u
			u_n[k] += step_size
			err = linalg.norm(u_n - u)
			u = u_n

		# center of the ellipse
		center = np.dot(P.T, u)

		# A matrix for the ellipse
		A = linalg.inv(np.dot(P.T, np.dot(np.diag(u), P)) -
		               np.array([[a * b for b in center] for a in center])) / d

		# Get the radii and rotation
		U, s, rotation = linalg.svd(A)
		radii = 1 / np.sqrt(s)

		return center, radii, rotation

	@staticmethod
	def getEllipsoidVolume(radii):
		"""Calculate the volume of the blob"""
		return 4/3 * np.pi *radii[0]*radii[1]*radii[2]

	@staticmethod
	def plot_ellipsoid(center, radii, rotation, ax=None, plot_axes=False, showSurface=False, alpha=0.5):
		"""Plot an ellipsoid"""
		color = tuple(np.random.randint(256, size=3) / 255)

		make_ax = ax is None
		if make_ax:
			fig = plt.figure()
			ax = fig.add_subplot(111, projection='3d')

		u = np.linspace(0, 2*np.pi, 100)
		v = np.linspace(0, np.pi, 100)

		# get cartesian coordinates corresponding to the spherical angles:
		x = radii[0] * np.outer(np.cos(u), np.sin(v))
		y = radii[1] * np.outer(np.sin(u), np.sin(v))
		z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

		# rotate accordingly
		for i, _ in enumerate(x):
			for j, _ in enumerate(x):
				[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center

		if plot_axes:
			axes = np.array([[radii[0], 0, 0],
			                 [0, radii[1], 0],
			                 [0, 0, radii[2]]])
			# rotate accordingly
			for i, axis in enumerate(axes):
				axes[i] = np.dot(axes[i], rotation)

			# plot axes
			for p in axes:
				X = np.linspace(-p[0], p[0], 100) + center[0]
				Y = np.linspace(-p[1], p[1], 100) + center[1]
				Z = np.linspace(-p[2], p[2], 100) + center[2]
				ax.plot(X, Y, Z, color=color)

		# Plot ellipsoid
		if showSurface:
			ax.plot_surface(x, y, z, rstride=3, cstride=3, color=color, alpha=alpha)
		else:
			ax.plot_wireframe(x, y, z, rstride=3, cstride=3, color=color, alpha=alpha)

		if make_ax:
			plt.show()

In [4]:
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

P1 = np.array([0.5,0,0.5])
P2 = np.array([2,-1,0])
P3 = np.array([1,1,-1])

e12 = 0.02
e13 = 0.04
e23 = 0.06
ez = 0.002

C1, C2, C3 = predict_u.compute_covariance_prism(P1, P2, P3, e12, e13, e23, ez)

In [5]:
%matplotlib notebook

# 1r 2g 3b
# 12:2 13:4 23:6
x1,y1,z1 = Ellipsoid_coordinates(C1, P1)
x2,y2,z2 = Ellipsoid_coordinates(C2, P2)
x3,y3,z3 = Ellipsoid_coordinates(C3, P3)
plot_three_Ellipsoids(x1,y1,z1,x2,y2,z2,x3,y3,z3,P1,P2,P3)

<IPython.core.display.Javascript object>

# Process static experiments with point data

In [6]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

# Important: set path to deployment data
path = "../data/20221103-2/"

# Important: change path to rosbag
file = "/home/maxime/data/FR2023/20221103/20221103_2_inter_prism.bag"

Tf = []
Tf.append(np.identity(4))
Tf.append(np.identity(4))
Tf.append(np.identity(4))
trajectory_trimble_1, trajectory_trimble_2, trajectory_trimble_3, time_trimble_1, time_trimble_2, time_trimble_3 = theodo_u.read_rosbag_theodolite_with_tf(file, Tf)
trimble_1 = np.array(trajectory_trimble_1).T
trimble_2 = np.array(trajectory_trimble_2).T
trimble_3 = np.array(trajectory_trimble_3).T

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


Number of data for theodolites: [4896 4873 4962]
Bad measures: 1444


In [7]:
%matplotlib notebook
plot_3d = 0   # Variable to plot in 3d, 0 = 2D, 1 = 3D
save_pdf = 0   # Variable to save file in pdf
file_pdf = path + "figures/Raw_data_TS_with_GCP_calibration.png"   # file name to save plot in pdf
theodo_p.plot_trajectories_prism(3, trimble_1, trimble_2, trimble_3, np.identity(4), np.identity(4), np.identity(4), plot_3d, save_pdf, file_pdf, 1)

<IPython.core.display.Javascript object>

In [8]:
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

P_nm_1_list = theodo_u.find_not_moving_points(trimble_1, 0.01)

Groupe_index_1_list = []
start_1 = P_nm_1_list[0]
list = []
for i in P_nm_1_list:
	if np.linalg.norm(trimble_1.T[start_1][0:3]-trimble_1.T[i][0:3])<0.01:
		list.append(i)
	else:
		if(len(list)>10):
			Groupe_index_1_list.append(list)
		list = []
		start_1 = i
if len(list)>0:
	Groupe_index_1_list.append(list)

In [9]:
%matplotlib notebook
plt.figure(0)
for i in Groupe_index_1_list:
	plt.scatter(trimble_1.T[i][:,0],trimble_1.T[i][:,1])
plt.show()

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


<IPython.core.display.Javascript object>

In [10]:
#print(Groupe_index_1_list[0])
#print(trimble_1.T[Groupe_index_1_list[0]][:,0])

print(len(Groupe_index_1_list))
list_number = 31
print(len(trimble_1.T[Groupe_index_1_list[list_number]]))

Average_p1_0_x = np.mean(trimble_1.T[Groupe_index_1_list[list_number]][:,0])
Average_p1_0_y = np.mean(trimble_1.T[Groupe_index_1_list[list_number]][:,1])
Average_p1_0_z = np.mean(trimble_1.T[Groupe_index_1_list[list_number]][:,2])


32
53


In [11]:
%matplotlib notebook
plt.figure(0)
# plt.plot(trimble_1.T[Groupe_index_1_list[0]][:,0], 'b')
# plt.axhline(Average_p1_0_x, color='b', linestyle='dashed')
plt.plot(trimble_1.T[Groupe_index_1_list[list_number]][:,1], 'r')
plt.axhline(Average_p1_0_y, color='r', linestyle='dashed')
# plt.plot(trimble_1.T[Groupe_index_1_list[0]][:,2], 'g')
# plt.axhline(Average_p1_0_z, color='g', linestyle='dashed')
plt.show()

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


<IPython.core.display.Javascript object>

In [12]:
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

P_1_0 = trimble_1.T[Groupe_index_1_list[list_number]]
# print(P_1_0.T[0:3])
Cov_1_0 = np.cov(P_1_0.T[0:3])
w, v = np.linalg.eig(Cov_1_0)
print(Cov_1_0)
print((w)**0.5)
print(v)

[[ 6.88316270e-06 -1.46306770e-06  2.29996473e-06]
 [-1.46306770e-06  1.21346321e-06 -8.84352638e-07]
 [ 2.29996473e-06 -8.84352638e-07  7.06879612e-06]]
[0.00309909 0.00217298 0.00091611]
[[ 0.69200119 -0.68964817  0.21335362]
 [-0.19394707  0.10706983  0.97515157]
 [ 0.69535523  0.71618535  0.05966279]]


# Process raw data


In [13]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

# Important: set path to deployment data
path = "../data/20221103-2/"

# Important: change path to rosbag
file = "/home/maxime/data/FR2023/20221103/20221103_2_inter_prism.bag"

_, _, _, _, _, _, distance_1, distance_2, distance_3, azimuth_1, azimuth_2, azimuth_3, elevation_1, elevation_2, elevation_3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(file)

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


Number of data for theodolites: [4896 4873 4962]
Bad measures: 1444


In [64]:
theodo_u = importlib.reload(theodo_u)

Groupe_dist_1_list, Mean_list_1, index_list_1 = theodo_u.split_static_distance(distance_1, 0.01, 10)
Groupe_dist_2_list, Mean_list_2, index_list_2 = theodo_u.split_static_distance(distance_2, 0.01, 10)
Groupe_dist_3_list, Mean_list_3, index_list_3 = theodo_u.split_static_distance(distance_3, 0.01, 10)

Groupe_azim_1_list = theodo_u.split_angle_form_index_list(azimuth_1, index_list_1)
Groupe_azim_2_list = theodo_u.split_angle_form_index_list(azimuth_2, index_list_2)
Groupe_azim_3_list = theodo_u.split_angle_form_index_list(azimuth_3, index_list_3)

Groupe_elev_1_list = theodo_u.split_angle_form_index_list(elevation_1, index_list_1)
Groupe_elev_2_list = theodo_u.split_angle_form_index_list(elevation_2, index_list_2)
Groupe_elev_3_list = theodo_u.split_angle_form_index_list(elevation_3, index_list_3)

In [83]:
xs1 = range(len(Mean_list_1))
labels1 = Mean_list_1
def format_fn1(tick_val, tick_pos):
    if int(tick_val) in xs1:
        return labels1[int(tick_val)]
    else:
        return ''

xs2 = range(len(Mean_list_2))
labels2 = Mean_list_2
def format_fn2(tick_val, tick_pos):
    if int(tick_val) in xs2:
        return labels2[int(tick_val)]
    else:
        return ''

xs3 = range(len(Mean_list_3))
labels3 = Mean_list_3
def format_fn3(tick_val, tick_pos):
    if int(tick_val) in xs3:
        return labels3[int(tick_val)]
    else:
        return ''

In [91]:
%matplotlib notebook
from matplotlib.ticker import MaxNLocator

fig, axs = plt.subplots(3, 3, layout=None, figsize=(12, 8))

bp = axs[0,0].boxplot(Groupe_dist_1_list, showfliers=False)
axs[0,0].xaxis.set_major_formatter(format_fn1)
axs[0,0].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,0].set_xlabel('Distance  [m]')
axs[0,0].set_ylabel('Distance [mm]')
axs[0,0].set_ylim([-5,5])

bp = axs[1,0].boxplot(Groupe_azim_1_list, showfliers=False)
axs[1,0].xaxis.set_major_formatter(format_fn1)
axs[1,0].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,0].set_xlabel('Distance [m]')
axs[1,0].set_ylabel('Azimuth [rad]')
axs[1,0].set_ylim([-2.2e-5,2e-5])

bp = axs[2,0].boxplot(Groupe_elev_1_list, showfliers=False)
axs[2,0].xaxis.set_major_formatter(format_fn1)
axs[2,0].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,0].set_xlabel('Distance to prism 1 [m]')
axs[2,0].set_ylabel('Elevation [rad]')
axs[2,0].set_ylim([-4.5e-5,4.5e-5])
######################################################################
bp = axs[0,1].boxplot(Groupe_dist_2_list, showfliers=False)
axs[0,1].xaxis.set_major_formatter(format_fn2)
axs[0,1].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,1].set_xlabel('Distance [m]')
#axs[0,1].set_ylabel('Error static \ndistance [mm]')
axs[0,1].set_ylim([-5,5])

bp = axs[1,1].boxplot(Groupe_azim_2_list, showfliers=False)
axs[1,1].xaxis.set_major_formatter(format_fn2)
axs[1,1].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,1].set_xlabel('Distance [m]')
#axs[1,1].set_ylabel('Error static \nazimuth [rad]')
axs[1,1].set_ylim([-2.2e-5,2e-5])

bp = axs[2,1].boxplot(Groupe_elev_2_list, showfliers=False)
axs[2,1].xaxis.set_major_formatter(format_fn2)
axs[2,1].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,1].set_xlabel('Distance to prism 2 [m]')
#axs[2,1].set_ylabel('Error static \nelevation [rad]')
axs[2,1].set_ylim([-4.5e-5,4.5e-5])
######################################################################
bp = axs[0,2].boxplot(Groupe_dist_3_list, showfliers=False)
axs[0,2].xaxis.set_major_formatter(format_fn3)
axs[0,2].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,2].set_xlabel('Distance [m]')
#axs[0,2].set_ylabel('Error static \ndistance [mm]')
axs[0,2].set_ylim([-5,5])

bp = axs[1,2].boxplot(Groupe_azim_3_list, showfliers=False)
axs[1,2].xaxis.set_major_formatter(format_fn3)
axs[1,2].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,2].set_xlabel('Distance [m]')
#axs[1,2].set_ylabel('Error static \nazimuth [rad]')
axs[1,2].set_ylim([-2.2e-5,2e-5])

bp = axs[2,2].boxplot(Groupe_elev_3_list, showfliers=False)
axs[2,2].xaxis.set_major_formatter(format_fn3)
axs[2,2].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,2].set_xlabel('Distance to prism 3 [m]')
#axs[2,2].set_ylabel('Error static \nelevation [rad]')
axs[2,2].set_ylim([-4.5e-5,4.5e-5])

plt.tight_layout()
plt.savefig(path + "figures/Uncertainty_static_position.jpg")
# show plot
plt.show()

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


<IPython.core.display.Javascript object>